In [ ]:

import numpy as np
import matplotlib.pyplot as plt

# --- MODEL IMPLEMENTATION ---------------------------------------------------
def agriculture_model_rk4(init_state, steps, params, dt=0.01):
    """
    4th-order Runge-Kutta integrator for the agro-ecosystem ODEs.
    """
    init_state = np.asarray(init_state, dtype=float)
    a, b, n, e, d, f, h, p, m, s = (np.asarray(params[k], dtype=float) for k in
                                   ('a', 'b', 'n', 'e', 'd', 'f', 'h', 'p', 'm', 's'))
    X = np.empty((steps, 5), dtype=float)
    X[0] = init_state

    alpha = params.get('alpha', 1.0)
    T = params.get('T', 100)
    z = params.get('z', 0.1)

    def rhs(state, t):
        DC, PP, PC, SC, TC = state
        time = t * dt
        a1_seasonal = 0.5 + alpha * np.cos(2 * np.pi * time / T)

        dDC = DC * (z * (d[1] * PP*PC + d[2] * PC *SC + d[3] * SC*TC + n[4] * TC) * (1 - b[0] * DC) )
        dPP = PP * (a1_seasonal * (1 - b[1] * PP) - d[1] * PC - n[1] - h[1] - p[1]
                    + f[1] * DC + s[2] * PC)
        dPC = PC * (e[2] * PP - d[2] * SC - n[2] - p[2])
        dSC = SC * (e[3] * PC - d[3] * TC - n[3] )
        dTC = TC * (e[4] * SC - n[4])

        return np.array([dDC, dPP, dPC, dSC, dTC], dtype = float)

    for t in range(steps - 1):
        k1 = dt * rhs(X[t], t)
        k2 = dt * rhs(X[t] + 0.5 * k1, t)
        k3 = dt * rhs(X[t] + 0.5 * k2, t)
        k4 = dt * rhs(X[t] + k3, t)
        X[t + 1] = X[t] + (k1 + 2 * k2 + 2 * k3 + k4) / 6.0

        if not np.isfinite(X[t + 1]).all():
            raise FloatingPointError(f"Overflow at step {t + 1}; try smaller dt.")
        X[t + 1] = np.maximum(X[t + 1], 0.0)

    return X


# --- PLOTTING FUNCTION ------------------------------------------------------
def plot_biomass(t, data, title, labels, period = 100., delta = 0.01, ylim=None):

    colors = ['#4575b4', '#91bfdb', '#fee090', '#fc8d59', '#d73027']  # colorblind friendly colors

    plt.figure(figsize=(12, 6))
    for k in range(5):
        plt.plot(t, data[:, k], label=labels[k], linewidth=2, color=colors[k])

    plt.xlabel(f"Time Steps (1 Year = {period / delta:.0f} Steps)", fontsize=14, weight='bold')
    plt.ylabel('Biomass', fontsize=14, weight='bold')
    plt.title(title, fontsize=16, weight='bold')
    plt.xticks(fontsize=12, weight='bold')
    plt.yticks(fontsize=12, weight='bold')
    if ylim:
        plt.ylim(ylim)
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.minorticks_on()
    plt.tick_params(axis='both', which='major', length=6, width=2)
    plt.tick_params(axis='both', which='minor', length=3, width=1)
    plt.legend(fontsize=11, loc='upper right')
    plt.tight_layout()
    plt.show()



## Alpha Sensitivity

In [ ]:
# --- PARAMETER SET ----------------------------------------------------------

def get_params(test_param = 0.1, period = 100, h_dr = 0.1, p_dr = 0.01, herbicide_on=True, pesticide_on=True):
    h_val = h_dr if herbicide_on else 0.0
    p_val = p_dr if pesticide_on else 0.0
    return dict(
        a=np.array([0., 1., 0., 0., 0.]),
        b=np.array([1., 1., 0., 0., 0.]),
        n=np.array([0., 0., 0.0, 0.0, 0.05]),
        e=np.array([0., 0., 0.3, 0.2, 0.12]),
        d=np.array([0., 0.2, 0.73, 0.338, 0.]),
        f=np.array([0., 0.1, 0., 0., 0.]),
        h=np.array([0., h_val, 0., 0., 0.]),
        p=np.array([0., 0.0, p_val, 0., 0.]),
        m=np.array([0., 0., 0., 0., 0.]),
        s=np.array([0., 0., 0.048, 0., 0.0]),
        alpha= test_param,
        T=period,
        z=0.1
    )

# --- SIMULATION + PLOTS -----------------------------------------------------
init = [0.3, 0.5, 0.15, 0.1, 0.1]
steps = 100000       # reduce for quicker plotting, or increase for longer dynamics
dt = 0.01
labels = ['Decomposers', 'Primary producers', 'Primary consumers',
          'Secondary consumers', 'Tertiary consumers']
t = np.arange(steps)

# Run simulations
T = 100.
h = 0.05
p = 0.05
y_bounds = (0., 1.5)
param_name = "alpha"
test_value = [0.025, 0.1, 0.25, 0.5]

# Plot each scenario
for test in test_value:
  out_test = agriculture_model_rk4(init, steps, get_params(test_param = test, period = T, h_dr = h, p_dr = p,herbicide_on = False, pesticide_on = False), dt)
  plot_biomass(t, out_test, f"{steps / (T / dt):.0f} Year Scenario: No Herbicide, No Pesticide, {param_name} = {test}", labels, period = T, delta = dt, ylim = y_bounds)

## Herbicide and Pesticide Sensitivity

In [ ]:

# --- PARAMETER SET ----------------------------------------------------------

def get_params(test_param = 0.1, period = 100, h_dr = 0.1, p_dr = 0.01, herbicide_on=True, pesticide_on=True):
    h_val = test_param if herbicide_on else 0.0
    p_val = p_dr if pesticide_on else 0.0
    return dict(
        a=np.array([0., 1., 0., 0., 0.]),
        b=np.array([1., 1., 0., 0., 0.]),
        n=np.array([0., 0., 0.0, 0.0, 0.05]),
        e=np.array([0., 0., 0.3, 0.2, 0.12]),
        d=np.array([0., 0.2, 0.73, 0.338, 0.]),
        f=np.array([0., 0.1, 0., 0., 0.]),
        h=np.array([0., h_val, 0., 0., 0.]),
        p=np.array([0., 0.0, p_val, 0., 0.]),
        m=np.array([0., 0., 0., 0., 0.]),
        s=np.array([0., 0., 0.048, 0., 0.0]),
        alpha= 0.25,
        T=period,
        z=0.1
    )

# --- SIMULATION + PLOTS -----------------------------------------------------
init = [0.3, 0.5, 0.15, 0.1, 0.1]
steps = 100000       # reduce for quicker plotting, or increase for longer dynamics
dt = 0.01
labels = ['Decomposers', 'Primary producers', 'Primary consumers',
          'Secondary consumers', 'Tertiary consumers']
t = np.arange(steps)

# Run simulations
T = 100.
h = 0.05
p = 0.05
y_bounds = (0., 1.5)
param_name = "alpha"
test_value = [0.01, 0.05, 0.1, 0.15]

# Plot each scenario
for test in test_value:
  out_test = agriculture_model_rk4(init, steps, get_params(test_param = test, period = T, h_dr = h, p_dr = p,herbicide_on = True, pesticide_on = False), dt)
  plot_biomass(t, out_test, f"{steps / (T / dt):.0f} Year Scenario: Herbicide Only ({test} = Death Rate)", labels, period = T, delta = dt, ylim = y_bounds)